# 深層強化学習講義 演習

## 目次
**課題：様々な強化学習アルゴリズムを実装して学習させてみましょう**
1. シミュレータ環境の構築
2. 価値に基づく手法 (Value-based Methods)
  - 2.1 テーブル解法
    - 2.1.1 SARSA
    - 2.1.2 Q-Learning
  - 2.2 NNによる価値関数の近似
    - 2.2.1 Deep Q-Network（DQN）
3. 方策に基づく手法 (Policy-based Methods)
  - 3.1 方策勾配法：方策の近似
    - 3.1.1 REINFORCE
  - 3.2 Actor-Critic
    - 3.2.1 Actor-Criticの実装例
  - 3.3 連続値行動空間
    - 3.3.1 連続値行動空間の環境の例：pendulum-v0
    - 3.3.2 Deep Deterministic Policy Gradient (DDPG)

**今回の講義の演習に関する注意事項**
- 実行時にメモリをたくさん使用するので，colabだと途中で実行が止まってしまうことがあります．
- その場合は，ランタイムを再起動して，以下の3つのセルを実行してから目的のセルを再実行してください．

In [ ]:
# Colab上では以下を実行してください
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install JSAnimation

In [ ]:
# Colab上では以下を実行してください
from pyvirtualdisplay import Display
pydisplay = Display(visible=0, size=(400, 300))
pydisplay.start()

In [ ]:
import numpy as np
import copy
from collections import deque
import gym
from gym import wrappers
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical, Normal
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt

from IPython import display
from JSAnimation.IPython_display import display_animation
from IPython.display import HTML

## 1. シミュレータ環境の構築
OpenAI Gymと呼ばれる，強化学習のシミュレータのライブラリを用いて環境を作成します．
- 今回は，`CartPole-v0`と呼ばれる，台車に振子がついた環境を利用します（[詳細](https://github.com/openai/gym/wiki/CartPole-v0)）．
    - 状態空間：4つの連続値
        1. カートの座標
        1. カートの速度
        1. ポールの角度
        1. ポールの角速度
    - 行動空間：1つの離散値．
      - 左に押す(0) または 右に押す(1)
    - 以下の終端条件のいずれかを満たすとエピソードが終了します．
      - 棒の角度が±12°より傾いたとき
      - 台車の位置が±2.4の範囲を逸脱したとき
      - エピソードの長さが200ステップに達したとき
    - 報酬：終端条件に達するまで常に+1

環境を作成してランダムな行動を取ってみましょう．

In [ ]:
env = gym.make('CartPole-v0')  # シミュレータ環境の構築
frames = []
for episode in range(5):
    state = env.reset()  # エピソードを開始（環境の初期化）
    env.render()  # シミュレータ画面の出力
    screen = env.render(mode='rgb_array')  # notebook上での結果の可視化用
    frames.append(screen)
    done = False
    while not done:
        action = env.action_space.sample()  # 行動をランダムに選択
        next_state, reward, done, _ = env.step(action)  # 行動を実行し、次の状態、 報酬、 終端か否かの情報を取得
        env.render()
        screen = env.render(mode='rgb_array')
        frames.append(screen)
env.close()  # 画面出力の終了

以下のセルを実行すると，notebook上で結果が可視化できます．

In [ ]:
# 結果の確認
plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0), dpi=72)
patch = plt.imshow(frames[0])
plt.axis('off')
  
def animate(i):
    patch.set_data(frames[i])
    
anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)
HTML(anim.to_jshtml())

**やってみよう！（余力のある人向けの追加課題）**
1. OpenAI Gymには，他にも様々なシミュレータ・ゲーム環境が用意されています．それらの環境をインポートして，ランダムな行動をとったときの結果を可視化してみましょう．
    - 環境の一覧は，OpenAI Gymの[webサイト](https://gym.openai.com/envs/)や[Github wiki](https://github.com/openai/gym/wiki/Table-of-environments)に掲載されています．
      - ライブラリを追加でインストールすることが必要な環境がありますが，今回その詳細については扱いません．

In [ ]:
# LET'S TRY

## 2.価値に基づく手法 (Value-based Methods)
価値に基づく手法では，価値に基づいて行動を決定する方策を利用します．
- 方策$\pi$として，小さい確率$\epsilon$で一様な確率でランダムな行動を選択し，それ以外は最もQ値（の推定値）が最も高い行動を選択する，**ε-greedy方策**が用いられることが多いです．
  - ランダムな行動を選択することで，探索を促進するために利用されます．

今回は，価値に基づく代表的な手法として，**SARSA**と**Q-learning**を扱います．
- どちらも以下の再帰的な更新により，Q関数を推定します，$$Q^{\pi}(s_t,a_t) \leftarrow Q^{\pi}(s_t,a_t)+\alpha \delta_t$$$$\delta_t = y_t - Q^{\pi}(s_t,a_t)$$
  - これは，Q値を目標値$y_t$に向かって更新する操作になっています．
  - $\alpha$：**学習率**（ステップサイズ）
  - $\delta_t$：**TD誤差** (temporal difference error)
  - $y_t$：**TDターゲット**
SARSAとQ-learningでTDターゲット$y_t$の求め方が異なります．$$y^{SARSA}_t = r_{t+1}+\gamma Q^{\pi}(s_{t+1},a_{t+1})$$$$y^{Q-learning}_t = r_{t+1}+\gamma \max_{a'}Q^{\pi}(s_{t+1},a')$$
  - $\gamma$：**割引率**

### 2.1 テーブル解法
- cartpoleは連続値をとる状態空間の問題設定です．テーブル解法では，本来は連続値として表現される状態空間を適当な間隔で区切って離散化します．
- ここでは，状態空間の各次元を6分割して， $6^4\times2=2692$個の値を持つ表としてQ関数を表現します．

まずは，そのために便利な関数を作っておきましょう．

In [ ]:
def bins(clip_min, clip_max, num):
    return np.linspace(clip_min, clip_max, num+1)[1:-1]

# 状態を離散化して対応するインデックスを返す関数（binの上限・下限はcartpole環境固有のものを用いています）
def discretize_state(observation, num_discretize):
    c_pos, c_v, p_angle, p_v = observation
    discretized = [
        np.digitize(c_pos, bins=bins(-2.4, 2.4, num_discretize)), 
        np.digitize(c_v, bins=bins(-3.0, 3.0, num_discretize)),
        np.digitize(p_angle, bins=bins(-0.5, 0.5, num_discretize)),
        np.digitize(p_v, bins=bins(-2.0, 2.0, num_discretize))
    ]
    return sum([x*(num_discretize**i) for i, x in enumerate(discretized)])

#### 2.1.1 SARSA
SARSAのQ関数に関する要素は以下の通りです．
- 価値の更新：$Q^{\pi}(s_t,a_t) \leftarrow Q^{\pi}(s_t,a_t)+\alpha \delta_t$
- TD誤差：$\delta_t = y^{SARSA}_t - Q^{\pi}(s_t,a_t)$
- TDターゲット：$y^{SARSA}_t = r_{t+1}+\gamma Q^{\pi}(s_{t+1},a_{t+1})$
それぞれ代入するとSARSAの価値の更新式は$$Q^{\pi}(s_t,a_t) \leftarrow Q^{\pi}(s_t,a_t)+\alpha \left(r_{t+1}+\gamma Q^{\pi}(s_{t+1},a_{t+1}) -Q^{\pi}(s_t,a_t)\right)$$となります，

方策は**ε-greedy方策**を用いてみましょう．
  - ランダムな行動をとる$\epsilon$は，定数でないこともよくあります．
  - 学習が進むごとに減衰させることで，学習初期は探索を促し，終盤は活用を促すように設計することも多いです．
  - 学習した方策の性能のテストをするときは，常にQ値が最大の行動をとるようにします．

**補足説明**
- SARSAでは，Q値の更新に利用する行動$a$は，方策$\pi$から得られたものであるので，**on-policy（方策オン）**の手法です．
  - on-policyの手法では，方策を制御に用いる一方で，同じ方策を用いて方策の価値を推定します．
  - Q-learning（off-policy）の価値の更新式と比較すると違いが明確になります．
- この実装ではエピソードが途中で終了した場合はペナルティを本来の報酬から引いています
  - 後に述べるように．テーブルを用いてQ関数を表現した場合，学習効率が悪く不安定になりがちなためです．
  - このような，学習を容易にするための追加的な報酬の設計を**reward shaping**といいます．

In [ ]:
class SarsaAgent:
    def __init__(self, num_state, num_action, num_discretize, gamma=0.99, alpha=0.5, max_initial_q=0.1):
        self.num_action = num_action
        self.gamma = gamma  # 割引率
        self.alpha = alpha  # 学習率
        # Qテーブルを作成し乱数で初期化
        self.qtable = np.random.uniform(low=-max_initial_q, high=max_initial_q, size=(num_discretize**num_state, num_action)) 
    
    # Qテーブルを更新
    def update_qtable(self, state, action, reward, next_state, next_action):
        self.qtable[state, action] += self.alpha*(reward+self.gamma*self.qtable[next_state, next_action]-self.qtable[state, action])
    
    # Q値が最大の行動を選択
    def get_greedy_action(self, state):
        action = np.argmax(self.qtable[state])
        return action
    
    # ε-greedyに行動を選択
    def get_action(self, state, episode):
        epsilon = 0.7 * (1/(episode+1))  # ここでは0.5から減衰していくようなεを設定
        if epsilon <= np.random.uniform(0,1):
            action = self.get_greedy_action(state)
        else:
            action = np.random.choice(self.num_action)
        return action

In [ ]:
# 各種設定
num_episode =1200  # 学習エピソード数
penalty = 10  # 途中でエピソードが終了したときのペナルティ
num_discretize = 6  # 状態空間の分割数
# ログ用の設定
episode_rewards = []
num_average_epidodes = 10

# エージェントの学習
env = gym.make('CartPole-v0')
max_steps = env.spec.max_episode_steps  # エピソードの最大ステップ数
agent = SarsaAgent(env.observation_space.shape[0], env.action_space.n, num_discretize)
for episode in range(num_episode):
    observation = env.reset()  # envからは4次元の連続値の観測が帰ってくる
    state = discretize_state(observation, num_discretize)  # 観測の離散化（状態のインデックスを取得）
    action = agent.get_action(state, episode)  #  行動を選択
    episode_reward = 0
    for t in range(max_steps):
        observation, reward, done, _ = env.step(action)
        # もしエピソードの途中で終了してしまったらペナルティを加える
        if done and t < max_steps - 1:
            reward = - penalty
        episode_reward += reward
        next_state = discretize_state(observation, num_discretize)
        next_action = agent.get_action(next_state, episode)
        agent.update_qtable(state, action, reward, next_state, next_action)  # Q値の表を更新
        state, action = next_state, next_action
        if done:
            break
    episode_rewards.append(episode_reward)
    if episode % 50 == 0:
        print("Episode %d finished | Episode reward %f" % (episode, episode_reward))
            
# 学習途中の累積報酬の移動平均を表示
moving_average = np.convolve(episode_rewards, np.ones(num_average_epidodes)/num_average_epidodes, mode='valid')
plt.plot(np.arange(len(moving_average)),moving_average)
plt.title('SARSA: average rewards in %d episodes' % num_average_epidodes)
plt.xlabel('episode')
plt.ylabel('rewards')
plt.show()

env.close()

エピソードで得られた合計報酬が改善していき，方策が学習できていることがわかります．
- 注意：初期値などの設定より，学習が非常に不安定になる場合があります．

それでは，得られた方策をテストして結果の動画を表示してみましょう．

In [ ]:
# 最終的に得られた方策のテスト（可視化）
env = gym.make('CartPole-v0')
frames = []
for episode in range(5):
    observation = env.reset()
    state = discretize_state(observation, num_discretize)
    frames.append(env.render(mode='rgb_array'))
    done = False
    while not done:
        action = agent.get_greedy_action(state)
        next_observation, reward, done, _ = env.step(action)
        frames.append(env.render(mode='rgb_array'))
        state = discretize_state(next_observation, num_discretize)
env.close()

plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0), dpi=72)
patch = plt.imshow(frames[0])
plt.axis('off')
  
def animate(i):
    patch.set_data(frames[i])
    
anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)
HTML(anim.to_jshtml())

ランダムな行動を選択したときよりも，長い間立て続けられていると思います．

**やってみよう！（余力のある人向けの追加課題）**
1. 上記のプログラムには複数のハイパーパラメータがあります．それらを変化させて，学習曲線がどのように変化するかを観察してみましょう．
    - ハイパーパラメータの例
      - `num_discretize`：離散化する数
      - `gamma`：割引率
      - `alpha`：学習率
      - `penalty`：途中で学習が終了したときのペナルティの大きさ
      - `epsilon`：ε-greedy方策でのεの関数（例えば，定数にしてみる，線形に減衰させてみる，など）
      - `num_episode`：学習エピソード数
      - `max_initial_q`：Qテーブルの初期化に用いる乱数の最大値

In [ ]:
# LET'S TRY

#### 2.1.2 Q-Learning
Q-learningのQ関数に関する要素は以下の通りです．
- 価値の更新：$Q^{\pi}(s_t,a_t) \leftarrow Q^{\pi}(s_t,a_t)+\alpha \delta_t$
- TD誤差：$\delta_t = y^{Q-learning}_t - Q^{\pi}(s_t,a_t)$
- TDターゲット：$y^{Q-learning}_t = r_{t+1}+\gamma \max_{a'}Q^{\pi}(s_{t+1},a')$

それぞれ代入するとQ-learningの価値の更新式は$$Q^{\pi}(s_t,a_t) \leftarrow Q^{\pi}(s_t,a_t)+\alpha \left(r_{t+1}+\gamma \max_{a'}Q^{\pi}(s_{t+1},a') -Q^{\pi}(s_t,a_t)\right)$$となります，

**補足説明**
- Q-learningでは，Q値の更新に利用する行動$a$は，Q関数が最大となる行動を決定的に選択するので，**off-policy（方策オフ）**の手法です．
  - off-policyの手法では，学習時の制御に用いる方策（**挙動方策**）と，評価され改善される方策（**推定方策，ターゲット方策**）で異なるものを用います．

In [ ]:
class QLearningAgent:
    def __init__(self, num_state, num_action, num_discretize, gamma=0.99, alpha=0.5, max_initial_q=0.1):
        self.num_action = num_action
        self.gamma = gamma  # 割引率
        self.alpha = alpha  # 学習率
        # Qテーブルを作成し乱数で初期化
        self.qtable = np.random.uniform(low=-max_initial_q, high=max_initial_q, size=(num_discretize**num_state, num_action))

    # Qテーブルを更新
    def update_qtable(self, state, action, reward, next_state):
        self.qtable[state, action] \
            += self.alpha*(reward+self.gamma*self.qtable[next_state, np.argmax(self.qtable[next_state])] - self.qtable[state, action])
    
    # Q値が最大の行動を選択
    def get_greedy_action(self, state):
        action = np.argmax(self.qtable[state])
        return action
    
    # ε-greedyに行動を選択
    def get_action(self, state, episode):
        epsilon = 0.7 * (1/(episode+1))  # ここでは0.5から減衰していくようなεを設定
        if epsilon <= np.random.uniform(0,1):
            action = self.get_greedy_action(state)
        else:
            action = np.random.choice(self.num_action)
        return action

In [ ]:
# 各種設定
num_episode =1200  # 学習エピソード数
penalty = 10  # 途中でエピソードが終了したときのペナルティ
num_discretize = 6  # 状態空間の分割数

# ログ
episode_rewards = []
num_average_epidodes = 10

env = gym.make('CartPole-v0')
max_steps = env.spec.max_episode_steps  # エピソードの最大ステップ数

agent = QLearningAgent(env.observation_space.shape[0], env.action_space.n, num_discretize)

for episode in range(num_episode):
    observation = env.reset()  # envからは4次元の連続値の観測が返ってくる
    state = discretize_state(observation, num_discretize)  # 観測の離散化（状態のインデックスを取得）
    episode_reward = 0
    for t in range(max_steps):
        action = agent.get_action(state, episode)  #  行動を選択
        observation, reward, done, _ = env.step(action)
        # もしエピソードの途中で終了してしまったらペナルティを加える
        if done and t < max_steps - 1:
            reward = - penalty
        episode_reward += reward
        next_state = discretize_state(observation, num_discretize)
        agent.update_qtable(state, action, reward, next_state)  # Q値の表を更新
        state = next_state
        if done:
            break
    episode_rewards.append(episode_reward)
    if episode % 50 == 0:
        print("Episode %d finished | Episode reward %f" % (episode, episode_reward))

# 累積報酬の移動平均を表示
moving_average = np.convolve(episode_rewards, np.ones(num_average_epidodes)/num_average_epidodes, mode='valid')
plt.plot(np.arange(len(moving_average)),moving_average)
plt.title('Q-Learning: average rewards in %d episodes' % num_average_epidodes)
plt.xlabel('episode')
plt.ylabel('rewards')
plt.show()

env.close()

In [ ]:
# 最終的に得られた方策のテスト（可視化）
env = gym.make('CartPole-v0')
frames = []
for episode in range(5):
    observation = env.reset()
    state = discretize_state(observation, num_discretize)
    frames.append(env.render(mode='rgb_array'))
    done = False
    while not done:
        action = agent.get_greedy_action(state)
        next_observation, reward, done, _ = env.step(action)
        frames.append(env.render(mode='rgb_array'))
        state = discretize_state(next_observation, num_discretize)
env.close()

plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0), dpi=72)
patch = plt.imshow(frames[0])
plt.axis('off')
  
def animate(i):
    patch.set_data(frames[i])
    
anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)
HTML(anim.to_jshtml())

**やってみよう！（余力のある人向けの追加課題）**
1. SARSAのときと同様に，Q-learningにもハイパーパラメータが複数あります．それらを変化させた場合，学習曲線がどのように変化するかを観察してみましょう．

In [ ]:
# LET'S TRY

### 2.2 NNによる価値関数の近似
これまでは，連続値の状態空間を持つ問題に対して，価値関数に基づく手法で状態空間を離散化して，価値関数をテーブルとして表現していました．

ここでは，ニューラルネットワークを用いて価値関数を近似することを考えます
- 状態空間の離散化を行わずに連続値のまま扱うことができます

#### 2.2.1 Deep Q-Network（DQN）
DQN（Deep Q-Network）は，Q関数をNNによって近似した手法です．Q-learningのTD誤差は，$$\delta_t = r_{t+1}+\gamma \max_{a'}Q^{\pi}(s_{t+1},a') - Q^{\pi}(s_t,a_t)$$であるので，$r_{t+1}+\gamma \max_{a'}Q^{\pi}(s_{t+1},a')$と$Q^{\pi}(s_t,a_t)$の差の最小化をすればいいことがわかります．
- 今回の実装ではMSEを用いています．

**特徴**
- **経験リプレイ**（experience replay）の利用
  - **リプレイバッファ**（replay buffer）にこれまでの状態遷移を記録しておき，そこからサンプルすることでQ関数のミニバッチ学習をします．
- **固定したターゲットネットワーク**（fixed target Q-network）の利用
  - 学習する対象のQ関数$Q^{\pi}(s_t,a_t)$も，更新によって近づける目標値$r_{t+1}+\gamma \max_{a'}Q^{\pi}(s_{t+1},a')$もどちらも同じパラメータを持つQ関数を利用しています．そのため，そのまま勾配法による最適化を行うと，元のQ値と目標値の両方が更新されてしまうことになります．
  - これを避けるために，目標値$r_{t+1}+\gamma \max_{a'}Q^{\pi}(s_{t+1},a')$は固定した上でQ関数の最適化を行います．
    - 実装上は，目標値側のQ関数の勾配の情報を削除して数値として扱います．

**論文**
- [Human-level control through deep reinforcement learning （Nature版）](https://www.nature.com/articles/nature14236)
- [Playing Atari with Deep Reinforcement Learning （NIPS2013 workshop版）](https://arxiv.org/abs/1312.5602)


**補足説明**
- 深層強化学習には様々な実装の仕方がありますが，今回の講義では以下のモジュールに分けて実装しています．
  - 1つ目のセル：関数近似のためのニューラルネットワーク（とリプレイバッファ）
  - 2つ目のセル：エージェントの定義
  - 3つ目のセル：実際に環境と相互作用して学習を実行する部分
  - (4つ目のセル：学習したエージェントの結果の可視化）

In [ ]:
# Q関数の定義
class QNetwork(nn.Module):
    def __init__(self, num_state, num_action, hidden_size=16):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(num_state, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, num_action)

    def forward(self, x):
        h = F.elu(self.fc1(x))
        h = F.elu(self.fc2(h))
        h = F.elu(self.fc3(h))
        y = F.elu(self.fc4(h))
        return y

# リプレイバッファの定義
class ReplayBuffer:
    def __init__(self, memory_size):
        self.memory_size = memory_size
        self.memory = deque([], maxlen = memory_size)
    
    def append(self, transition):
        self.memory.append(transition)
    
    def sample(self, batch_size):
        batch_indexes = np.random.randint(0, len(self.memory), size=batch_size)
        states      = np.array([self.memory[index]['state'] for index in batch_indexes])
        next_states = np.array([self.memory[index]['next_state'] for index in batch_indexes])
        rewards     = np.array([self.memory[index]['reward'] for index in batch_indexes])
        actions     = np.array([self.memory[index]['action'] for index in batch_indexes])
        dones   = np.array([self.memory[index]['done'] for index in batch_indexes])
        return {'states': states, 'next_states': next_states, 'rewards': rewards, 'actions': actions, 'dones': dones}

In [ ]:
class DqnAgent:
    def __init__(self, num_state, num_action, gamma=0.99, lr=0.001, batch_size=32, memory_size=50000):
        self.num_state = num_state
        self.num_action = num_action
        self.gamma = gamma  # 割引率
        self.batch_size = batch_size  # Q関数の更新に用いる遷移の数
        self.qnet = QNetwork(num_state, num_action)
        self.target_qnet = copy.deepcopy(self.qnet)  # ターゲットネットワーク
        self.optimizer = optim.Adam(self.qnet.parameters(), lr=lr)
        self.replay_buffer = ReplayBuffer(memory_size)
    
    # Q関数を更新
    def update_q(self):
        batch = self.replay_buffer.sample(self.batch_size)
        q = self.qnet(torch.tensor(batch["states"], dtype=torch.float))
        targetq = copy.deepcopy(q.data.numpy())
        # maxQの計算
        maxq = torch.max(self.target_qnet(torch.tensor(batch["next_states"],dtype=torch.float)), dim=1).values
        # targetqのなかで，バッチのなかで実際に選択されていた行動 batch["actions"][i] に対応する要素に対して，Q値のターゲットを計算してセット
        # 注意：選択されていない行動のtargetqの値はqと等しいためlossを計算する場合には影響しない
        for i in range(self.batch_size):
            # 終端状態の場合はmaxQを0にしておくと学習が安定します（ヒント：maxq[i] * (not batch["dones"][i])）
            targetq[i, batch["actions"][i]] = batch["rewards"][i] + self.gamma * maxq[i] * (not batch["dones"][i]) 
        self.optimizer.zero_grad()
        # lossとしてMSEを利用
        loss = nn.MSELoss()(q, torch.tensor(targetq))
        loss.backward()
        self.optimizer.step()
        # ターゲットネットワークのパラメータを更新
        self.target_qnet = copy.deepcopy(self.qnet)
    
    # Q値が最大の行動を選択
    def get_greedy_action(self, state):
        state_tensor = torch.tensor(state, dtype=torch.float).view(-1, self.num_state)
        action = torch.argmax(self.qnet(state_tensor).data).item()
        return action
    
    # ε-greedyに行動を選択
    def get_action(self, state, episode):
        epsilon = 0.7 * (1/(episode+1))  # ここでは0.5から減衰していくようなεを設定
        if epsilon <= np.random.uniform(0,1):
            action = self.get_greedy_action(state)
        else:
            action = np.random.choice(self.num_action)
        return action

In [ ]:
# 各種設定
num_episode = 300  # 学習エピソード数
memory_size = 50000  # replay bufferの大きさ
initial_memory_size = 500  # 最初に貯めるランダムな遷移の数

# ログ
episode_rewards = []
num_average_epidodes = 10

env = gym.make('CartPole-v0')
max_steps = env.spec.max_episode_steps  # エピソードの最大ステップ数

agent = DqnAgent(env.observation_space.shape[0], env.action_space.n, memory_size=memory_size)

# 最初にreplay bufferにランダムな行動をしたときのデータを入れる
state = env.reset()
for step in range(initial_memory_size):
    action = env.action_space.sample() # ランダムに行動を選択        
    next_state, reward, done, _ = env.step(action)
    transition = {
        'state': state,
        'next_state': next_state,
        'reward': reward,
        'action': action,
        'done': int(done)
    }
    agent.replay_buffer.append(transition)
    state = env.reset() if done else next_state

for episode in range(num_episode):
    state = env.reset()  # envからは4次元の連続値の観測が返ってくる
    episode_reward = 0
    for t in range(max_steps):
        action = agent.get_action(state, episode)  # 行動を選択
        next_state, reward, done, _ = env.step(action)
        episode_reward += reward
        transition = {
            'state': state,
            'next_state': next_state,
            'reward': reward,
            'action': action,
            'done': int(done)
        }
        agent.replay_buffer.append(transition)
        agent.update_q()  # Q関数を更新
        state = next_state
        if done:
            break
    episode_rewards.append(episode_reward)
    if episode % 20 == 0:
        print("Episode %d finished | Episode reward %f" % (episode, episode_reward))

# 累積報酬の移動平均を表示
moving_average = np.convolve(episode_rewards, np.ones(num_average_epidodes)/num_average_epidodes, mode='valid')
plt.plot(np.arange(len(moving_average)),moving_average)
plt.title('DQN: average rewards in %d episodes' % num_average_epidodes)
plt.xlabel('episode')
plt.ylabel('rewards')
plt.show()

env.close()

In [ ]:
# 最終的に得られた方策のテスト（可視化）
env = gym.make('CartPole-v0')
frames = []
for episode in range(5):
    state = env.reset()
    frames.append(env.render(mode='rgb_array'))
    done = False
    while not done:
        action = agent.get_greedy_action(state)
        state, reward, done, _ = env.step(action)
        frames.append(env.render(mode='rgb_array'))
env.close()

plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0), dpi=72)
patch = plt.imshow(frames[0])
plt.axis('off')
  
def animate(i):
    patch.set_data(frames[i])
    
anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)
HTML(anim.to_jshtml())

**やってみよう！（余力のある人向けの追加課題）**
1. ハイパーパラメータを変化させた場合，学習曲線がどのように変化するかを観察してみましょう．
  - とくに，価値関数の近似器として用いたニューラルネットワーク（`QNetwork`）の構造（層の数や中間層のユニットの数）を変更するとどうなるでしょうか．
  - また，リプレイバッファの大きさ（`memory_size`）を変えるとどうなるでしょうか．

In [ ]:
# LET'S TRY

## 3.方策に基づく手法 (Policy-based Methods) 


### 3.1 方策勾配法：方策の近似
- 方策勾配法では，方策を直接ニューラルネットワークで関数近似します．
  - 実際には，方策と価値関数の両方を関数近似するactor-criticの手法（3.2項）を用いることも多いです．
- 方策$\pi_{\theta}(\cdot)$に対して目的関数$$\mathcal{J}(\pi_{\theta}) = \mathbb{E_{\pi_{\theta}}}\left[f_{\pi_{\theta}}(\cdot)\right]$$を最**大**化する$\theta$を探索する手法です．
  - $f_{\pi_{\theta}}(\cdot)$：方策の良さを測る指標（詳しくは[Schulman 2015](https://arxiv.org/abs/1506.02438)参照）
    - **REINFORCE**：収益（割引報酬和）$R_t=\sum_{k=t}^{T} \gamma^{k-t}r_{k+1}$を用いる場合
    - **actor-critic**：状態価値や行動価値（の推定値）を用いる場合
  - 実装上は目的関数の最小化を考えることが多いので，符号が反転します．
- このとき**方策勾配**（policy gradient）は，
$$\nabla_{\theta}\mathcal{J}(\pi_{\theta}) = \mathbb{E_{\pi_{\theta}}}\left[\nabla_{\theta}\log\pi_{\theta} \cdot f_{\pi_{\theta}}(\cdot)\right]$$と定義されます．

#### 3.1.1 REINFORCE
- REINFORCEは，方策の良さの指標$f_{\pi_{\theta}}(\cdot)$として収益$$\begin{align}R_t &= r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + \cdots +\gamma^{T-t-1} r_T \\ &= \sum_{k=t}^{T} \gamma^{k-t}r_{k+1}\end{align}$$を用いる手法です．つまり，$$f_{\pi_{\theta}}(\cdot)=R_t$$とすることで，方策勾配は，
$$\nabla_{\theta}\mathcal{J}(\pi_{\theta}) = \frac{1}{T}\sum_{t=1}^{T}\mathbb{E_{\pi_{\theta}}}\left[\nabla_{\theta}\log\pi_{\theta}(a_t|s_t)\cdot R_t\right]$$となります．
- REINFORCEでは，収益$R_t$の値は分散は一般に大きいため，方策勾配の分散が大きくなり，学習が不安定になりやすいことが知られています．
  - そのため，収益$R_t$から**ベースライン**(baseline)$b(s)$を引く，つまり，$$f_{\pi_{\theta}}(\cdot)=R_t-b(s)$$とすることで，学習の安定化を図ることがよくあります．
    - ベースライン$b(s)$を引いても，方策勾配の期待値には影響しないことが知られています．
- REINFORCEでは，確率的な方策を採用しています．
  - 今回のような離散の行動空間では，NNがそれぞれの行動の選択確率を出力するように実装することがよくあります．
    - 確率的な行動の選択は[カテゴリカル分布](https://pytorch.org/docs/stable/distributions.html#categorical)$P(x=k ; \mathbf{p})=p_{k}$からサンプリングを行う実装が多いです．
    - テスト時には，出力された確率値が最大の行動を選びます（greedy方策）．

**論文**
- [Simple statistical gradient-following algorithms for connectionist reinforcement learning](https://link.springer.com/article/10.1007/BF00992696)

In [ ]:
# 方策のネットワークの定義
class PolicyNetwork(nn.Module):
    def __init__(self, num_state, num_action, hidden_size=16):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(num_state, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_action)
    
    def forward(self, x):
        h = F.elu(self.fc1(x))
        h = F.elu(self.fc2(h))
        action_prob = F.softmax(self.fc3(h), dim=-1)
        return action_prob

In [ ]:
class ReinforceAgent:
    def __init__(self, num_state, num_action, gamma=0.99, lr=0.001):
        self.num_state = num_state
        self.gamma = gamma  # 割引率
        self.pinet = PolicyNetwork(num_state, num_action)
        self.optimizer = optim.Adam(self.pinet.parameters(), lr=lr)
        self.memory = []  # 報酬とそのときの行動選択確率のtupleをlistで保存
    
    # 方策を更新
    def update_policy(self):
        R = 0
        loss = 0
        # エピソード内の各ステップの収益を後ろから計算
        for r, prob in self.memory[::-1]:
            R = r + self.gamma * R
            loss -= torch.log(prob) * R
        loss = loss/len(self.memory)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
    
    # softmaxの出力が最も大きい行動を選択
    def get_greedy_action(self, state):
        state_tensor = torch.tensor(state, dtype=torch.float).view(-1, self.num_state)
        action_prob = self.pinet(state_tensor.data).squeeze()
        action = torch.argmax(action_prob.data).item()
        return action
    
    # カテゴリカル分布からサンプリングして行動を選択
    def get_action(self, state):
        state_tensor = torch.tensor(state, dtype=torch.float).view(-1, self.num_state)
        action_prob = self.pinet(state_tensor.data).squeeze()
        action = Categorical(action_prob).sample().item()
        return action, action_prob[action]
    
    def add_memory(self, r, prob):
        self.memory.append((r, prob))
    
    def reset_memory(self):
        self.memory = []

In [ ]:
# 各種設定
num_episode = 600  # 学習エピソード数
# penalty = 10  # 途中でエピソードが終了したときのペナルティ

# ログ
episode_rewards = []
num_average_epidodes = 10

env = gym.make('CartPole-v0')
max_steps = env.spec.max_episode_steps  # エピソードの最大ステップ数

agent = ReinforceAgent(env.observation_space.shape[0], env.action_space.n)

for episode in range(num_episode):
    state = env.reset()  # envからは4次元の連続値の観測が返ってくる
    episode_reward = 0
    for t in range(max_steps):
        action, prob = agent.get_action(state)  #  行動を選択
        next_state, reward, done, _ = env.step(action)
#         # もしエピソードの途中で終了してしまったらペナルティを加える
#         if done and t < max_steps - 1:
#             reward = - penalty
        episode_reward += reward
        agent.add_memory(reward, prob)
        state = next_state
        if done:
            agent.update_policy()
            agent.reset_memory()
            break
    episode_rewards.append(episode_reward)
    if episode % 20 == 0:
        print("Episode %d finished | Episode reward %f" % (episode, episode_reward))

# 累積報酬の移動平均を表示
moving_average = np.convolve(episode_rewards, np.ones(num_average_epidodes)/num_average_epidodes, mode='valid')
plt.plot(np.arange(len(moving_average)),moving_average)
plt.title('REINFORCE: average rewards in %d episodes' % num_average_epidodes)
plt.xlabel('episode')
plt.ylabel('rewards')
plt.show()

env.close()

In [ ]:
# 最終的に得られた方策のテスト（可視化）
env = gym.make('CartPole-v0')
frames = []
for episode in range(5):
    state = env.reset()
    frames.append(env.render(mode='rgb_array'))
    done = False
    while not done:
        action = agent.get_greedy_action(state)
        state, reward, done, _ = env.step(action)
        frames.append(env.render(mode='rgb_array'))
env.close()

plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0), dpi=72)
patch = plt.imshow(frames[0])
plt.axis('off')
  
def animate(i):
    patch.set_data(frames[i])
    
anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)
HTML(anim.to_jshtml())

**やってみよう！（余力のある人向けの追加課題）**
1. ハイパーパラメータ・ネットワークを変更した場合，学習曲線がどのように変化するかを観察してみましょう．
2. 報酬から適当なベースラインを引いて実験してみましょう．
  - 現在までの経験の平均報酬をベースラインとして利用する方法が一般的です．

In [ ]:
# LET'S TRY

### 3.2 Actor-Critic：方策と価値関数の近似

#### 3.2.1 Actor-Criticの実装例
- 方策（actor）と価値関数（critic）の両方をニューラルネットワークで近似します．
  - 今回の実装では，状態価値関数をモデル化して，$$f_{\pi_{\theta}}(\cdot)=R_t-V^{\pi_{\theta}}(s)$$として実装してみます．
    - この実装は，REINFORCEにおけるベースライン$b(s)$を$V^{\pi_{\theta}}(s)$とした場合とも解釈できます．
  
**補足説明**
- とくに，$$f_{\pi_{\theta}}(\cdot) = Q^{\pi_{\theta}}(s, a) - V^{\pi_{\theta}}(s)$$とした場合の$f_{\pi_{\theta}}(\cdot)$は**アドバンテージ**（advantage）$A^{\pi_{\theta}}(s,a)$と呼ばれています．
  - これは，状態$s$における行動$a$の相対的な良さを表しています．
  - この組み合わせ以外の場合にも，[一般化アドバンテージ（generalized advantage estimation, GAE）](https://arxiv.org/abs/1506.02438)として単に「アドバンテージ」ということがあります．
- 実装上，actorとcriticのネットワークの共有はよく行われます．

In [ ]:
# actorとcriticのネットワーク（一部の重みを共有しています）
class ActorCriticNetwork(nn.Module):
    def __init__(self, num_state, num_action, hidden_size=16):
        super(ActorCriticNetwork, self).__init__()
        self.fc1 = nn.Linear(num_state, hidden_size)
        self.fc2a = nn.Linear(hidden_size, num_action)  # actor独自のlayer
        self.fc2c = nn.Linear(hidden_size, 1)  # critic独自のlayer
    
    def forward(self, x):
        h = F.elu(self.fc1(x))
        action_prob = F.softmax(self.fc2a(h), dim=-1)
        state_value = self.fc2c(h)
        return action_prob, state_value

In [ ]:
class ActorCriticAgent:
    def __init__(self, num_state, num_action, gamma=0.99, lr=0.001):
        self.num_state = num_state
        self.gamma = gamma  # 割引率
        self.acnet = ActorCriticNetwork(num_state, num_action)
        self.optimizer = optim.Adam(self.acnet.parameters(), lr=lr)
        self.memory = []  # （報酬，行動選択確率，状態価値）のtupleをlistで保存
        
    # 方策を更新
    def update_policy(self):
        R = 0
        actor_loss = 0
        critic_loss = 0
        for r, prob, v in self.memory[::-1]:
            R = r + self.gamma * R
            advantage = R - v
            actor_loss -= torch.log(prob) * advantage
            critic_loss += F.smooth_l1_loss(v, torch.tensor(R))
        actor_loss = actor_loss/len(self.memory)
        critic_loss = critic_loss/len(self.memory)
        self.optimizer.zero_grad()
        loss = actor_loss + critic_loss
        loss.backward()
        self.optimizer.step()
    
    # softmaxの出力が最も大きい行動を選択
    def get_greedy_action(self, state):
        state_tensor = torch.tensor(state, dtype=torch.float).view(-1, self.num_state)
        action_prob, _ = self.acnet(state_tensor.data)
        action = torch.argmax(action_prob.squeeze().data).item()
        return action
        
    # カテゴリカル分布からサンプリングして行動を選択
    def get_action(self, state):
        state_tensor = torch.tensor(state, dtype=torch.float).view(-1, self.num_state)
        action_prob, state_value = self.acnet(state_tensor.data)
        action_prob, state_value = action_prob.squeeze(), state_value.squeeze()
        action = Categorical(action_prob).sample().item()
        return action, action_prob[action], state_value
    
    def add_memory(self, r, prob, v):
        self.memory.append((r, prob, v))
    
    def reset_memory(self):
        self.memory = []

In [ ]:
# 各種設定
num_episode = 1200  # 学習エピソード数
# penalty = 10  # 途中でエピソードが終了したときのペナルティ

# ログ
episode_rewards = []
num_average_epidodes = 10

env = gym.make('CartPole-v0')
max_steps = env.spec.max_episode_steps  # エピソードの最大ステップ数

agent = ActorCriticAgent(env.observation_space.shape[0], env.action_space.n)

for episode in range(num_episode):
    state = env.reset()  # envからは4次元の連続値の観測が返ってくる
    episode_reward = 0
    for t in range(max_steps):
        action, prob, state_value = agent.get_action(state)  #  行動を選択
        next_state, reward, done, _ = env.step(action)
#         # もしエピソードの途中で終了してしまったらペナルティを加える
#         if done and t < max_steps - 1:
#             reward = - penalty
        episode_reward += reward
        agent.add_memory(reward, prob, state_value)
        state = next_state
        if done:
            agent.update_policy()
            agent.reset_memory()
            break
    episode_rewards.append(episode_reward)
    if episode % 50 == 0:
        print("Episode %d finished | Episode reward %f" % (episode, episode_reward))

# 累積報酬の移動平均を表示
moving_average = np.convolve(episode_rewards, np.ones(num_average_epidodes)/num_average_epidodes, mode='valid')
plt.plot(np.arange(len(moving_average)),moving_average)
plt.title('Actor-Critic: average rewards in %d episodes' % num_average_epidodes)
plt.xlabel('episode')
plt.ylabel('rewards')
plt.show()

env.close()

In [ ]:
# 最終的に得られた方策のテスト（可視化）
env = gym.make('CartPole-v0')
frames = []
for episode in range(5):
    state = env.reset()
    frames.append(env.render(mode='rgb_array'))
    terminal = False
    while not terminal:
        action = agent.get_greedy_action(state)
        state, reward, terminal, _ = env.step(action)
        frames.append(env.render(mode='rgb_array'))
env.close()

plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0), dpi=72)
patch = plt.imshow(frames[0])
plt.axis('off')
  
def animate(i):
    patch.set_data(frames[i])
    
anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)
HTML(anim.to_jshtml())

**やってみよう！（余力のある人向けの追加課題）**
1. ハイパーパラメータ・ネットワークを変更した場合，学習曲線がどのように変化するかを観察してみましょう．
  - とくに，criticのネットワークを変更するとどうなるでしょうか．
1. Q値（行動価値）も推定して，アドバンテージを利用したactor-criticアルゴリズムを実装してみましょう．

In [ ]:
# LET'S TRY

### 3.3 連続値行動空間
最後に，ニューラルネットワークによって価値関数や方策の関数近似をしたactor-critic手法を，連続値の行動空間を持つ問題に利用してみましょう．
- 古典的には，REINFORCEで出力層に仮定した分布をカテゴリカル分布から正規分布（ガウス分布）に変更することで，連続値を出力する実装がよく利用されてきました．
- 近年では，[DDPG](https://arxiv.org/abs/1509.02971)（Deep Deterministic Policy Gradient）や，[SAC](https://arxiv.org/abs/1801.01290)（Soft Actor-Critc）と呼ばれる手法が用いられることが多いです．


#### 3.3.1 連続値行動空間の環境の例：pendulum-v0
- OpenAI Gymには，連続値行動空間の環境のシミュレータも含まれています．
- ここでは`pendulum-v0`という，軸の角速度を制御して振子を立てる問題を解いてみましょう（[詳細](https://github.com/openai/gym/wiki/Pendulum-v0)）．
    - 状態空間：3つの連続値
        1. $\cos(\theta)$：振子の向き$\theta$のコサイン $[-1.0,1.0]$
        1. $\sin(\theta)$：振子の向き$\theta$のサイン $[-1.0,1.0]$
        1. $\dot{\theta}$：振子の角速度 $[-8.0,8.0]$
    - 行動空間：1つの連続値．
      - $a$：関節（中心）にかける力 $[-2.0,2.0]$
    - エピソードの長さが200ステップに達したときにエピソードが終了します．
    - 報酬：終端条件に達するまで常に$-(\theta^2 + 0.1\times\dot{theta}^2 + 0.001\times a^2)$
    
環境を作成してランダムな行動を取ってみましょう．

In [ ]:
env = gym.make('Pendulum-v0')  # シミュレータ環境の構築
frames = []
for episode in range(3):
    state = env.reset()  # エピソードを開始（環境の初期化）
    frames.append(env.render(mode='rgb_array'))
    done = False
    while not done:
        action = env.action_space.sample()  # 行動をランダムに選択
        next_state, reward, done, _ = env.step(action)  # 行動を実行し、次の状態、 報酬、 終端か否かの情報を取得
        frames.append(env.render(mode='rgb_array'))
env.close()  # 画面出力の終了

# 結果の確認
plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0), dpi=72)
patch = plt.imshow(frames[0])
plt.axis('off')
  
def animate(i):
    patch.set_data(frames[i])
    
anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)
HTML(anim.to_jshtml())

#### 3.3.2 Deep Deterministic Policy Gradient (DDPG)

DDPG（Deep Deterministic Policy Gradient）は，actor（方策）とcritic（行動価値関数）の両方をニューラルネットワークで近似した手法です．

全体として，**連続値の行動空間に対してDQNを用いた手法**になっています．
- 連続値の行動空間に対して$\arg\max_{a'}Q_{\phi}^{\pi}(s_{t},a')$を求めるのは難しいので，方策もNNでパラメータ化することで解決しています．
- そのためにactor$\pi_{\theta}$に**決定論的な方策**を用いています．
  - なお，方策$\pi_{\theta}$が決定論的な場合，$\mu_{\theta}$と明示的に表記する文献もあります．

以上の設定によって，criticの学習はDQNと同様に，TD誤差$$\delta_t = r_{t+1}+\gamma Q_{\phi}^{\pi_{\theta}}(s_{t+1},\pi_{\theta}(s_t)) - Q_{\phi}^{\pi_{\theta}}(s_t,a_t)$$の最**小**化をすれば良いことがわかります．

一方，actorはQ関数$Q_{\phi}\left(s, \pi_{\theta}(s)\right)$の値が最**大**になるように方策勾配法を用いて学習します．つまり，actorの方策勾配は$$\nabla_{\theta}\mathcal{J}(\pi_{\theta})=\nabla_{\theta} Q_{\phi}\left(s, \pi_{\theta}(s)\right)$$となります．

実装では，これらのcriticとactorの更新を交互に繰り返して最適化を行います．


**論文**
- [Continuous control with deep reinforcement learning](https://arxiv.org/abs/1509.02971)

In [ ]:
# actorのネットワーク
class ActorNetwork(nn.Module):
    def __init__(self, num_state, action_space, hidden_size=16):
        super(ActorNetwork, self).__init__()
        self.action_mean = torch.tensor(0.5*(action_space.high+action_space.low), dtype=torch.float)
        self.action_halfwidth = torch.tensor(0.5*(action_space.high-action_space.low), dtype=torch.float)
        self.fc1 = nn.Linear(num_state, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, action_space.shape[0])

    def forward(self, s):
        h = F.elu(self.fc1(s))
        h = F.elu(self.fc2(h))
        a = self.action_mean + self.action_halfwidth*torch.tanh(self.fc3(h))
        return a

# criticのネットワーク（状態と行動を入力にしてQ値を出力）
class CriticNetwork(nn.Module):
    def __init__(self, num_state, action_space, hidden_size=16):
        super(CriticNetwork, self).__init__()
        self.action_mean = torch.tensor(0.5*(action_space.high+action_space.low), dtype=torch.float)
        self.action_halfwidth = torch.tensor(0.5*(action_space.high-action_space.low), dtype=torch.float)
        self.fc1 = nn.Linear(num_state+action_space.shape[0], hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, action_space.shape[0])

    def forward(self, s, a):
        a = (a-self.action_mean)/self.action_halfwidth
        h = F.elu(self.fc1(torch.cat([s,a],1)))
        h = F.elu(self.fc2(h))
        q = self.fc3(h)
        return q

# リプレイバッファの定義
class ReplayBuffer:
    def __init__(self, memory_size):
        self.memory_size = memory_size
        self.memory = deque([], maxlen = memory_size)
    
    def append(self, transition):
        self.memory.append(transition)
    
    def sample(self, batch_size):
        batch_indexes = np.random.randint(0, len(self.memory), size=batch_size)
        states      = np.array([self.memory[index]['state'] for index in batch_indexes])
        next_states = np.array([self.memory[index]['next_state'] for index in batch_indexes])
        rewards     = np.array([self.memory[index]['reward'] for index in batch_indexes])
        actions     = np.array([self.memory[index]['action'] for index in batch_indexes])
        dones   = np.array([self.memory[index]['done'] for index in batch_indexes])
        return {'states': states, 'next_states': next_states, 'rewards': rewards, 'actions': actions, 'dones': dones}

In [ ]:
class DdpgAgent:
    def __init__(self, observation_space, action_space, gamma=0.99, lr=1e-3, batch_size=32, memory_size=50000):
        self.num_state = observation_space.shape[0]
        self.num_action = action_space.shape[0]
        self.state_mean = 0.5*(observation_space.high + observation_space.low)
        self.state_halfwidth = 0.5*(observation_space.high - observation_space.low)
        self.gamma = gamma  # 割引率
        self.batch_size = batch_size
        self.actor = ActorNetwork(self.num_state, action_space)
        self.actor_target = copy.deepcopy(self.actor)  # actorのターゲットネットワーク
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=lr)
        self.critic = CriticNetwork(self.num_state, action_space)
        self.critic_target = copy.deepcopy(self.critic)  # criticのターゲットネットワーク
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=lr)
        self.replay_buffer = ReplayBuffer(memory_size)
    
    # 連続値の状態を[-1,1]の範囲に正規化
    def normalize_state(self, state):
        state = (state-self.state_mean)/self.state_halfwidth
        return state
    
    # リプレイバッファからサンプルされたミニバッチをtensorに変換
    def batch_to_tensor(self, batch):
        states = torch.tensor([self.normalize_state(s) for s in batch["states"]], dtype=torch.float)
        actions = torch.tensor(batch["actions"], dtype=torch.float)
        next_states = torch.tensor([self.normalize_state(s) for s in batch["next_states"]], dtype=torch.float)
        rewards = torch.tensor(batch["rewards"], dtype=torch.float)
        return states, actions, next_states, rewards
    
    # actorとcriticを更新
    def update(self):
        batch = self.replay_buffer.sample(self.batch_size)
        states, actions, next_states, rewards = self.batch_to_tensor(batch)
        # criticの更新
        target_q = (rewards + self.gamma*self.critic_target(next_states, self.actor_target(next_states)).squeeze()).data
        q = self.critic(states, actions).squeeze()
        critic_loss = F.mse_loss(q, target_q)
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()
        # actorの更新
        actor_loss = -self.critic(states, self.actor(states)).mean()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        # ターゲットネットワークのパラメータを更新
        self.critic_target = copy.deepcopy(self.critic)
        self.actor_target = copy.deepcopy(self.actor)
    
    # Q値が最大の行動を選択
    def get_action(self, state):
        state_tensor = torch.tensor(self.normalize_state(state), dtype=torch.float).view(-1, self.num_state)
        action = self.actor(state_tensor).view(self.num_action)
        return action

In [ ]:
# 各種設定
num_episode = 250  # 学習エピソード数（学習に時間がかかるので短めにしています）
memory_size = 50000  # replay bufferの大きさ
initial_memory_size = 1000  # 最初に貯めるランダムな遷移の数
# ログ用の設定
episode_rewards = []
num_average_epidodes = 10

env = gym.make('Pendulum-v0')
max_steps = env.spec.max_episode_steps  # エピソードの最大ステップ数
agent = DdpgAgent(env.observation_space, env.action_space, memory_size=memory_size)

# 最初にreplay bufferにランダムな行動をしたときのデータを入れる
state = env.reset()
for step in range(initial_memory_size):
    action = env.action_space.sample() # ランダムに行動を選択 
    next_state, reward, done, _ = env.step(action)
    transition = {
        'state': state,
        'next_state': next_state,
        'reward': reward,
        'action': action,
        'done': int(done)
    }
    agent.replay_buffer.append(transition)
    state = env.reset() if done else next_state
print('%d Data collected' % (initial_memory_size))

for episode in range(num_episode):
    state = env.reset()  # envからは3次元の連続値の観測が返ってくる
    episode_reward = 0
    for t in range(max_steps):
        action = agent.get_action(state).data.numpy()  #  行動を選択
        next_state, reward, done, _ = env.step(action)
        episode_reward += reward
        transition = {
            'state': state,
            'next_state': next_state,
            'reward': reward,
            'action': action,
            'done': int(done)
        }
        agent.replay_buffer.append(transition)
        agent.update()  # actorとcriticを更新
        state = next_state
        if done:
            break
    episode_rewards.append(episode_reward)
    if episode % 20 == 0:
        print("Episode %d finished | Episode reward %f" % (episode, episode_reward))

# 累積報酬の移動平均を表示
moving_average = np.convolve(episode_rewards, np.ones(num_average_epidodes)/num_average_epidodes, mode='valid')
plt.plot(np.arange(len(moving_average)),moving_average)
plt.title('DDPG: average rewards in %d episodes' % num_average_epidodes)
plt.xlabel('episode')
plt.ylabel('rewards')
plt.show()

env.close()

In [ ]:
# 最終的に得られた方策のテスト（可視化）
env = gym.make('Pendulum-v0')
max_steps = env.spec.max_episode_steps  # エピソードの最大ステップ数
frames = []
for episode in range(3):
    state = env.reset()
    frames.append(env.render(mode='rgb_array'))
    for t in range(max_steps):
        action = agent.get_action(state).detach().numpy()
        state, reward, terminal, _ = env.step(action)
        frames.append(env.render(mode='rgb_array'))
env.close()

plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0), dpi=72)
patch = plt.imshow(frames[0])
plt.axis('off')
  
def animate(i):
    patch.set_data(frames[i])
    
anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)
HTML(anim.to_jshtml())

**やってみよう！（余力のある人向けの追加課題）**
1. ハイパーパラメータ・ネットワークを変更した場合，学習曲線がどのように変化するかを観察してみましょう．
1. `pendulum-v0`以外の連続値行動空間のタスクを解いてみましょう．
  - OpenAI Gymで利用できる環境の一覧は，[webサイト](https://gym.openai.com/envs/)や[Github wiki](https://github.com/openai/gym/wiki/Table-of-environments)に掲載されています．
  - おすすめの環境
    - `MountainCarContinuous-v0`：車を山の頂上に到達させるタスク．入力の次元も小さく（2次元）で比較的簡単．
    - 以下の環境は，`box2d`と`gym[Box2D]`を追加でインストールする必要があります（notebook上では`!pip install box2d gym[Box2D]`で入ります）．
      - `LunarLanderContinuous-v2`：宇宙船を指定された位置に着陸させるタスク．
      - `BipedalWalker-v3`：2足歩行ロボットを歩かせるタスク．入力次元が24次元で少し難しい．
      - `BipedalWalkerHardcore-v3`：上記環境をもっと難しくしたタスク（障害物がある）．
1. 今回の講義で紹介されたもの以外の連続値行動空間の深層強化学習手法を調べて実装してみましょう．既存の手法の課題として何が挙げられていて，それに対してどのような工夫がなされているでしょうか．
  - 近年の代表的な手法の例
    - [TRPO（Trust Region Policy Optimization）](https://arxiv.org/abs/1502.05477)
    - [PPO（Proximal Policy Optimization）](https://arxiv.org/abs/1707.06347)
    - [SAC（Soft Actor-Critic）](https://arxiv.org/abs/1801.01290)
    - [TD3（Twin-Delayed DDPG）](https://arxiv.org/abs/1802.09477)
    - [AWR（Advantage-Weighted Regression）](https://arxiv.org/abs/1910.00177)

In [ ]:
# LET'S TRY

**補足説明**
- 今回の演習では状態としてロボットの関節角や角速度などを用いる問題設定を考えましたが，CNNなどを用いて，画像を入力とした連続値制御をすることもできます．